# 1, Single image input

1, Loads an image into PIL format.
2, Converts a PIL Image instance to a Numpy array.
3, Expand the shape of the image array. In the example below, expand_dims convert shape(224,224,3) to shape(0,224,224,3). This is needed because the shape of input to model ResNet50 should be (None, 224,224,3)

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np




img_path = './elephant.jpg'  # image path   
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)        #(224,224,3)
x = np.expand_dims(x, axis=0)      #(0,224,224,3)
x = preprocess_input(x)

model = ResNet50(weights='imagenet')
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

# 2, Multiple images input

# Two ways to implement the multiple images input

## 2.1 Using glob -- a direct method
Firstly, use glob to do pathname pattern expansion.
Then, use for loop to load images in the same file.
 

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
import glob

file_path = './images/'
f_names = glob.glob(file_path + '*.jpg') #list

imgs = []
for i in range(len(f_names)):
    img = image.load_img(f_names[i], target_size = (224, 224))
    arr_img = image.img_to_array(img)
    arr_img = np.expand_dims(arr_img, axis=0)
    imgs.append(arr_img)

x = np.concatenate([x for x in imgs])

## 2.2 Generate a csv -- not a direct method

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications import xception


# function to convert image to array
def read_img(address, size):
    """Read and resize image.
    Returns Image as numpy array, by normalizing the values
    """
    img = image.load_img(address, target_size=size)
    img = image.img_to_array(img)
    return img

# function to convert labels to one hot encoding vector
def OneHotEncoded(y_train):
    y_t=np.zeros((len(y_train),Num_Class), dtype=int)
    for i,x in enumerate(y_train):
        y_t[i][int(x)]=1
    return y_t

import os
import csv
from shutil import copy2
directory_lists=os.listdir("./images/train")
labels = {}
data = {}

with open('./labels.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','class'])
    for i in range(0, len(directory_lists)):                        #'''directory_lists == files in /train == ['c0', 'c1', ... , 'c9']'''
        t = directory_lists[i][1:]
        imag=os.listdir("./images/train/"+str(directory_lists[i]))     #'''imag == files in /train/c# '''
        current_loc = "./images/train/"+str(directory_lists[i])+"/"    #'''./imgs/train/c#/'''
        for k in range(0, len(imag)):
            x=[current_loc+str(imag[k])]+[t]
            writer.writerow(x)

data_dir = './'
labels = pd.read_csv(join(data_dir, 'labels.csv'))

INPUT_SIZE = 299
POOLING = 'avg'
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
y_train= np.zeros((len(labels),), dtype='float32')

for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
    y_train[i]=int(labels['class'][i])
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

# 3 Using generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'images/train',
        target_size=(224, 224),
        batch_size=1,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'images/validation',
        target_size=(224, 224),
        batch_size=1,
        class_mode='binary')

model = ResNet50(weights = 'imagenet')
model.compile(loss='binary_crossentropy', optimizer='adam')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)